In [1]:
from keras.layers import *
from keras.models import Sequential
from keras.utils import to_categorical
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

Using TensorFlow backend.


In [2]:
model=Sequential()

model.add(Conv2D(32,(3,3), activation='relu',input_shape=(100,100,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation='relu',input_shape=(100,100,3)))
model.add(MaxPooling2D(2,2))
model.add(Conv2D(64,(3,3), activation='relu',input_shape=(100,100,3)))
model.add(MaxPooling2D(2,2))
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(2,activation='softmax'))
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 21, 21, 64)        36928     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 6400)             

In [3]:
# validation split

import os,shutil
folders=os.listdir("train")

if not os.path.isdir("val_images"):
    os.mkdir("val_images")
classes=["benign","malignant"]

for c in classes:
    p="val_images/"+c
    if not os.path.isdir(p):
        os.mkdir(p)
        
split=0.9
for f in os.listdir("train"):
    path="train/"+f
    imgs=os.listdir(path)
    
    split_size=int(split*len(imgs))
    files_to_move=imgs[split_size:]
    
    for img_f in files_to_move:
        src=path+"/"+img_f
        dest="val_images/"+f+"/"+img_f
        shutil.move(src,dest)


In [4]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

training_set = train_datagen.flow_from_directory("train/",
                                                 target_size = (100, 100),
                                                 batch_size = 32)

val_datagen = ImageDataGenerator(rescale = 1./255)
val_set = val_datagen.flow_from_directory("val_images/",
                                                 target_size = (100, 100),
                                                 batch_size = 32)
                                                 

Found 2135 images belonging to 2 classes.
Found 238 images belonging to 2 classes.


In [5]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [6]:
from  keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping

checkpoint=ModelCheckpoint("best_model.h5",monitor="val_loss", verbose=0, save_best_only=True, save_weights_only=False, mode='auto', period=1)
earlystop=EarlyStopping(monitor="val_accuracy", patience=4)

history=model.fit_generator(training_set,
                            epochs = 10,
                            steps_per_epoch=128,
                            validation_data=val_set,
                            validation_steps=8,
                            callbacks=[checkpoint,earlystop])

Epoch 1/10
128/128 [==============================] - 119s 926ms/step - loss: 0.5208 - accuracy: 0.7256 - val_loss: 0.4805 - val_accuracy: 0.7647
Epoch 2/10
128/128 [==============================] - 92s 717ms/step - loss: 0.4030 - accuracy: 0.8001 - val_loss: 0.2380 - val_accuracy: 0.7899
Epoch 3/10
128/128 [==============================] - 92s 721ms/step - loss: 0.3714 - accuracy: 0.8266 - val_loss: 0.2019 - val_accuracy: 0.7857
Epoch 4/10
128/128 [==============================] - 92s 721ms/step - loss: 0.3660 - accuracy: 0.8313 - val_loss: 0.5206 - val_accuracy: 0.7815
Epoch 5/10
128/128 [==============================] - 94s 731ms/step - loss: 0.3547 - accuracy: 0.8352 - val_loss: 0.3055 - val_accuracy: 0.8151
Epoch 6/10
128/128 [==============================] - 92s 721ms/step - loss: 0.3436 - accuracy: 0.8412 - val_loss: 0.3115 - val_accuracy: 0.8151
Epoch 7/10
128/128 [==============================] - 94s 738ms/step - loss: 0.3133 - accuracy: 0.8519 - val_loss: 0.9244 - val_a

In [8]:
model.load_weights("best_model.h5")

In [10]:
# predictions
xtest=[]
testimg=image.load_img("1251.jpg",target_size=(100,100))
X=image.img_to_array(testimg)/255.0
xtest.append(X)
xtest=np.array(xtest)
ans=model.predict(xtest)
print(ans)

[[0.12407731 0.8759227 ]]


In [11]:
predicted_class=np.argmax(ans)
print(predicted_class)

1
